## ch11 XGBoost Hyperparameters
- https://github.com/mattharrison/effective_xgboost_book/blob/main/xgbcode.ipynb

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial upload: 2023.7.31</div>
<div style="text-align: right"> Last update: 2023.7.31</div>

In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats
import warnings; warnings.filterwarnings('ignore')
#plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

새롭게 라이브러리를 로드 하자.

In [2]:
import dtreeviz
from feature_engine import encoding, imputation
import numpy as np
import pandas as pd
from sklearn import base, compose, datasets, ensemble, \
    metrics, model_selection, pipeline, preprocessing, tree
import scikitplot
import xgboost as xgb
import yellowbrick.model_selection as ms
from yellowbrick import classifier

import urllib
import zipfile

import xg_helpers as xhelp

In [3]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
'kaggle-survey-2018.zip'
fname = 'kaggle-survey-2018.zip'
member_name = 'multipleChoiceResponses.csv'

In [4]:
raw = xhelp.extract_zip(url, fname, member_name)
## Create raw X and raw y
kag_X, kag_y = xhelp.get_rawX_y(raw, 'Q6')

In [5]:
## Split data
kag_X_train, kag_X_test, kag_y_train, kag_y_test = \
model_selection.train_test_split(
kag_X, kag_y, test_size=.3, random_state=42, stratify=kag_y)

In [6]:
## Transform X with pipeline
X_train = xhelp.kag_pl.fit_transform(kag_X_train)
X_test = xhelp.kag_pl.transform(kag_X_test)
## Transform y with label encoder
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(kag_y_train)
y_train = label_encoder.transform(kag_y_train)
y_test = label_encoder.transform(kag_y_test)
# Combined Data for cross validation/etc
X = pd.concat([X_train, X_test], axis='index')
y = pd.Series([*y_train, *y_test], index=X.index)

In [7]:
X.head()

,age,education,years_exp,compensation,python,r,sql,Q1_Male,Q1_Female,Q1_Prefer not to say,Q1_Prefer to self-describe,Q3_United States of America,Q3_India,Q3_China,major_cs,major_other,major_eng,major_stat
587,25,18.0,4.0,0,1,0,1,1,0,0,0,0,1,0,0,1,0,0
3065,22,16.0,1.0,10000,1,0,0,1,0,0,0,0,1,0,1,0,0,0
8435,22,18.0,1.0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
3110,40,20.0,3.0,125000,1,0,1,0,1,0,0,1,0,0,0,1,0,0
16372,45,12.0,5.0,100000,1,0,1,1,0,0,0,1,0,0,0,1,0,0


In [8]:
y.head()

587      1
3065     0
8435     0
3110     0
16372    1
dtype: int64

### 11.1 Hyperparameters

하이퍼파라미터는 다음과 같은 형태로 분류할 수 있다.
- tree creation,
- sampling, 
- categorical data, 
- ensembling, 
- regularization, 
- imbalanced data

tree 하이퍼파라미터는 다음과 같다.  
- max_depth=6 : 트리 깊이. 얼마나 많은 피처 상호작용을 할 수 있는지입니다. 클수록 더 복잡합니다(과적합 가능성이 높습니다). 각 레벨은 시간을 두 배로 늘립니다. 범위 [0, ∞].  
- max_leaves=0 : 트리의 잎 수입니다. 숫자가 클수록 더 복잡합니다. (Not tree_method='exact'에서 지원되지 않음). 범위 [0, ∞].  
- min_child_weight=1 : 자식에게 필요한 헤시안(무게)의 최소 합계입니다. 값이 클수록 값이 클수록 더 보수적입니다. 범위 [0,∞].  
- grow_policy="depthwise"' - (tree_method가 'hist', 'approx' 또는 'gpu_hist' 일때만 진행. 루트에 가장 가까운 노드를 분할합니다. "lossguide"(최대_뎁스=0으로 설정)로 설정하면 LightGBM 동작을 모방합니다.  
- tree_method="auto" : 히스토그램 버킷링을 사용하고 성능을 높이려면 "hist"를 사용합니다.
  - `auto`로 설정하면
    - 'exact' for small data
    – 'approx' for larger data
    – 'hist' for histograms (limit bins with max_bins (default 256))

sampling 하이퍼 파라미터. 이 파라미터들은 순차적으로 동작한다.  
- tree -> level -> 노드 순이다.  
예를 들어 colsample_bytree, colsample_bylevel 및 colsample_bynode가 모두 .5인 경우 트리는 원본 열의 12.5%(.5 * .5 * .5) 만 보게 됩니다.   

- colsample_bytree=1 : 각 트리에서 컬럼을 서브 샘플링합니다. 범위 (0,1]. [.5, 1] 를 권장
- colsample_bylevel=1 : 각 레벨에서 컬럼을 서브 샘플링합니다(트리 컬럼 colsample_bytree에서). 범위 (0,1]. [.5, 1]를 권장
- colsample_bynode=1 : 각 노드 분할에서 컬럼을 서브 샘플링합니다(트리 컬럼 colsample_bytree에서). 범위 (0,1]. [.5, 1]를 권장
- subsample=1 : 각 부스팅 라운드 전 학습 데이터(rows)의 샘플 비율. 범위 (0,1]. 더 보수적으로 만들려면 낮게 설정합니다. (1.0이 아닌 경우 모델은 확률적 경사 하강을 수행합니다. 즉, 모델에 약간의 무작위성이 있습니다.) [.5, 1] 범위를 권장.
- sampling_method='uniform' : 샘플링 방법. uniform 확률의 경우 'uniform'. gradient_based 샘플은 정규화된 그라데이션 절대값에 비례합니다(tree_method="gpu_hist"에서만 지원).

